## Deliverable 2. Create a Customer Travel Destinations Map.

In [53]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [54]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marawi,PH,7.9986,124.2928,68.76,92,83,4.32,broken clouds
1,1,Klaksvik,FO,62.2266,-6.5890,49.12,84,36,14.32,scattered clouds
2,2,Vila Franca Do Campo,PT,37.7167,-25.4333,73.81,86,49,3.76,scattered clouds
3,3,Broken Hill,AU,-31.9500,141.4333,42.44,82,0,12.08,clear sky
4,4,Ushuaia,AR,-54.8000,-68.3000,35.22,86,40,26.46,light snow


In [55]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [56]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [57]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Vila Franca Do Campo,PT,37.7167,-25.4333,73.81,86,49,3.76,scattered clouds
11,11,Nantucket,US,41.2835,-70.0995,80.71,92,100,34.87,light rain
12,12,Ribeira Grande,PT,38.5167,-28.7000,77.40,83,20,9.22,few clouds
15,15,Isangel,VU,-19.5500,169.2667,74.98,90,57,11.61,broken clouds
16,16,Albany,US,42.6001,-73.9662,82.78,56,40,1.99,scattered clouds
18,18,Hambantota,LK,6.1241,81.1185,78.15,87,100,11.23,heavy intensity rain
21,21,Atuona,PF,-9.8000,-139.0333,76.75,71,34,10.69,scattered clouds
23,23,Salalah,OM,17.0151,54.0924,76.44,97,100,6.64,overcast clouds
25,25,Kavieng,PG,-2.5744,150.7967,81.30,81,20,16.67,light rain
26,26,Bandarbeyla,SO,9.4942,50.8122,76.14,82,45,31.70,scattered clouds


In [60]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                328
City                   328
Country                328
Lat                    328
Lng                    328
Max Temp               328
Humidity               328
Cloudiness             328
Wind Speed             328
Current Description    328
dtype: int64

In [61]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#preferred_cities_df.replace('', n)

In [62]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Franca Do Campo,PT,73.81,scattered clouds,37.7167,-25.4333,
11,Nantucket,US,80.71,light rain,41.2835,-70.0995,
12,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,
15,Isangel,VU,74.98,broken clouds,-19.5500,169.2667,
16,Albany,US,82.78,scattered clouds,42.6001,-73.9662,
18,Hambantota,LK,78.15,heavy intensity rain,6.1241,81.1185,
21,Atuona,PF,76.75,scattered clouds,-9.8000,-139.0333,
23,Salalah,OM,76.44,overcast clouds,17.0151,54.0924,
25,Kavieng,PG,81.30,light rain,-2.5744,150.7967,
26,Bandarbeyla,SO,76.14,scattered clouds,9.4942,50.8122,


In [63]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Franca Do Campo,PT,73.81,scattered clouds,37.7167,-25.4333,Pestana Bahia Praia
11,Nantucket,US,80.71,light rain,41.2835,-70.0995,Jared Coffin House
12,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,Quinta da Meia Eira
15,Isangel,VU,74.98,broken clouds,-19.5500,169.2667,Tanna Lodge
16,Albany,US,82.78,scattered clouds,42.6001,-73.9662,
18,Hambantota,LK,78.15,heavy intensity rain,6.1241,81.1185,Bungalow 63
21,Atuona,PF,76.75,scattered clouds,-9.8000,-139.0333,Villa Enata
23,Salalah,OM,76.44,overcast clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
25,Kavieng,PG,81.30,light rain,-2.5744,150.7967,Nusa Island Retreat
26,Bandarbeyla,SO,76.14,scattered clouds,9.4942,50.8122,JABIR HOTEL


In [64]:
 import numpy as np
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Vila Franca Do Campo,PT,73.81,scattered clouds,37.7167,-25.4333,Pestana Bahia Praia
11,Nantucket,US,80.71,light rain,41.2835,-70.0995,Jared Coffin House
12,Ribeira Grande,PT,77.40,few clouds,38.5167,-28.7000,Quinta da Meia Eira
15,Isangel,VU,74.98,broken clouds,-19.5500,169.2667,Tanna Lodge
18,Hambantota,LK,78.15,heavy intensity rain,6.1241,81.1185,Bungalow 63
21,Atuona,PF,76.75,scattered clouds,-9.8000,-139.0333,Villa Enata
23,Salalah,OM,76.44,overcast clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
25,Kavieng,PG,81.30,light rain,-2.5744,150.7967,Nusa Island Retreat
26,Bandarbeyla,SO,76.14,scattered clouds,9.4942,50.8122,JABIR HOTEL
27,Bellingham,US,87.75,scattered clouds,48.7595,-122.4882,Best Western Heritage Inn


In [65]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [66]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))